In [52]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchsummary import summary

print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu126
True


In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)  

cuda


In [54]:
train_data = pd.read_csv('../data/train_data.csv')
test_data = pd.read_csv('../data/test_data.csv')

In [55]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3798 entries, 0 to 3797
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   day            3798 non-null   int64  
 1   pressure       3798 non-null   float64
 2   maxtemp        3798 non-null   float64
 3   temparature    3798 non-null   float64
 4   mintemp        3798 non-null   float64
 5   dewpoint       3798 non-null   float64
 6   humidity       3798 non-null   float64
 7   cloud          3798 non-null   float64
 8   sunshine       3798 non-null   float64
 9   winddirection  3798 non-null   float64
 10  windspeed      3798 non-null   float64
 11  rainfall       3798 non-null   int64  
dtypes: float64(10), int64(2)
memory usage: 356.2 KB


In [56]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             730 non-null    int64  
 1   day            730 non-null    int64  
 2   pressure       730 non-null    float64
 3   maxtemp        730 non-null    float64
 4   temparature    730 non-null    float64
 5   mintemp        730 non-null    float64
 6   dewpoint       730 non-null    float64
 7   humidity       730 non-null    float64
 8   cloud          730 non-null    float64
 9   sunshine       730 non-null    float64
 10  winddirection  730 non-null    float64
 11  windspeed      730 non-null    float64
dtypes: float64(10), int64(2)
memory usage: 68.6 KB


In [57]:
print(train_data.isnull().sum())  

day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    0
windspeed        0
rainfall         0
dtype: int64


In [58]:
print(test_data.isnull().sum())  

id               0
day              0
pressure         0
maxtemp          0
temparature      0
mintemp          0
dewpoint         0
humidity         0
cloud            0
sunshine         0
winddirection    0
windspeed        0
dtype: int64


### `Data Scaling`

In [59]:
ss = StandardScaler()
columns = list(train_data.columns)
columns.remove('rainfall')
columns.remove('day')
train_data[columns] = ss.fit_transform(train_data[columns])
test_data[columns] = ss.transform(test_data[columns])

In [60]:
train_data.head()

,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed,rainfall
0,7,0.631370,-0.986591,-0.688274,-0.477055,-0.166971,0.988286,0.965892,-1.012811,-0.548996,-0.410395,1
1,7,1.006851,-1.800977,-1.405593,-1.293287,-0.936347,1.838946,1.117212,-1.301544,-0.675864,0.124856,1
2,7,1.814504,-1.312213,-1.560690,-1.532184,-2.065857,-0.713033,-1.102144,0.877074,-0.422129,-0.307900,1
3,7,-0.083832,-1.547385,-1.231110,-1.074298,-0.667066,1.838946,1.218092,-1.301544,-0.548996,1.685055,1
4,7,1.418092,-0.968501,-1.114788,-1.412736,-2.051942,-1.847247,-1.203024,-0.356601,-0.802732,0.455117,0


In [61]:
train_data.day.unique()

array([  7,   8,   9,  10,  11,  12,  13,  14,  15,  16,  17,  18,  19,
        20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,  31,  32,
        33,  34,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  45,
        46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  57,  58,
        59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,
        72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,  84,
        85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
        98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110,
       111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123,
       124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136,
       137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149,
       150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 18

In [62]:
test_data.head()

,id,day,pressure,maxtemp,temparature,mintemp,dewpoint,humidity,cloud,sunshine,winddirection,windspeed
0,2190,1,1.006851,-1.655925,-1.618851,-1.910438,-1.032519,2.264276,1.520731,-1.301544,-0.675864,0.398175
1,2191,2,0.470449,-1.655925,-1.483141,-1.293287,-0.994050,2.406053,1.520731,-1.301544,-0.675864,1.650890
2,2192,3,1.793573,-2.795602,-2.665750,-2.567406,-2.186582,0.846510,1.369412,-1.301544,-0.802732,-0.444560
3,2193,4,1.614773,-1.095132,-1.328045,-1.412736,-2.071176,-0.713033,-1.203024,0.562093,-1.056467,3.393303
4,2194,5,1.489612,-1.909187,-2.006591,-3.164649,-3.071364,-1.705470,-1.001264,1.113309,-1.056467,-0.159852


In [63]:
X = train_data.drop(columns = ['rainfall'])
y = train_data['rainfall']

In [64]:
print(len(X.columns))
X.columns

11


Index(['day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint',
       'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed'],
      dtype='object')

In [65]:
y.head()

0    1
1    1
2    1
3    1
4    0
Name: rainfall, dtype: int64

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify = y)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train = torch.tensor(X_train.to_numpy(), dtype=torch.float32).to(device)
X_test = torch.tensor(X_test.to_numpy(), dtype=torch.float32).to(device)
y_train = torch.tensor(y_train.to_numpy(), dtype=torch.float32).to(device)
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32).to(device)

(2848, 11)
(950, 11)
(2848,)
(950,)


In [67]:
print(f"Train class distribution: {y_train.cpu().numpy().mean():.4f}")  # % of 1s
print(f"Test class distribution: {y_test.cpu().numpy().mean():.4f}")

Train class distribution: 0.5000
Test class distribution: 0.5000


In [69]:
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = TensorDataset(X_test, y_test)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=128, shuffle=False)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=True)

## 2. `Model Training`

In [78]:
class RainfallPredictor(nn.Module):
    def __init__(self, input_size):
        super(RainfallPredictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(), 
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = RainfallPredictor(input_size=X.shape[1]).to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

summary(model, (1, X.shape[1]))
# print(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 128]           1,536
              ReLU-2               [-1, 1, 128]               0
            Linear-3               [-1, 1, 128]          16,512
              ReLU-4               [-1, 1, 128]               0
            Linear-5               [-1, 1, 128]          16,512
              ReLU-6               [-1, 1, 128]               0
            Linear-7               [-1, 1, 128]          16,512
              ReLU-8               [-1, 1, 128]               0
            Linear-9                 [-1, 1, 1]             129
          Sigmoid-10                 [-1, 1, 1]               0
Total params: 51,201
Trainable params: 51,201
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.01
Params size (MB): 0.20
Estimated Tot

In [80]:
num_epochs = 100
model.to(device) 

for epoch in range(num_epochs):
    model.train()  
    epoch_loss = 0  
    
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float()

        optimizer.zero_grad()
        outputs = model(X_batch)
        
        loss = criterion(outputs, y_batch.unsqueeze(1)) 
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    avg_loss = epoch_loss / len(train_loader)
    
    # Validation Step
    if (epoch + 1) % 10 == 0:
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_val_batch, y_val_batch in val_loader:
                X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device).float()

                val_outputs = model(X_val_batch)

                val_loss += criterion(val_outputs.float(), y_val_batch.unsqueeze(1)).item()

        avg_val_loss = val_loss / len(val_loader)

        print(f"Validation Loss after Epoch {epoch+1}: {avg_val_loss:.4f}")

    print(f"Epoch [{epoch+1}/{num_epochs}], Avg Loss: {avg_loss:.4f}")

Epoch [1/100], Avg Loss: 0.3573
Epoch [2/100], Avg Loss: 0.3574
Epoch [3/100], Avg Loss: 0.3569
Epoch [4/100], Avg Loss: 0.3689
Epoch [5/100], Avg Loss: 0.3594
Epoch [6/100], Avg Loss: 0.3673
Epoch [7/100], Avg Loss: 0.3554
Epoch [8/100], Avg Loss: 0.3561
Epoch [9/100], Avg Loss: 0.3516
Validation Loss after Epoch 10: 0.4241
Epoch [10/100], Avg Loss: 0.3508
Epoch [11/100], Avg Loss: 0.3477
Epoch [12/100], Avg Loss: 0.3416
Epoch [13/100], Avg Loss: 0.3542
Epoch [14/100], Avg Loss: 0.3546
Epoch [15/100], Avg Loss: 0.3500
Epoch [16/100], Avg Loss: 0.3587
Epoch [17/100], Avg Loss: 0.3547
Epoch [18/100], Avg Loss: 0.3497
Epoch [19/100], Avg Loss: 0.3597
Validation Loss after Epoch 20: 0.4172
Epoch [20/100], Avg Loss: 0.3494
Epoch [21/100], Avg Loss: 0.3473
Epoch [22/100], Avg Loss: 0.3570
Epoch [23/100], Avg Loss: 0.3611
Epoch [24/100], Avg Loss: 0.3467
Epoch [25/100], Avg Loss: 0.3637
Epoch [26/100], Avg Loss: 0.3489
Epoch [27/100], Avg Loss: 0.3541
Epoch [28/100], Avg Loss: 0.3444
Epoch [

In [77]:
train_data.shape

(3798, 12)